In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import joblib
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/path/to/your/predictive_maintenance.csv')

label_encoders = {}
for col in ['Type', 'Product ID']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df = df.drop(['UDI'], axis=1)

X = df.drop(['Failure Type'], axis=1)
y = df['Failure Type']

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

failure_encoder = LabelEncoder()
y_res_encoded = failure_encoder.fit_transform(y_res)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res_encoded, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=failure_encoder.classes_))

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d',
            xticklabels=failure_encoder.classes_,
            yticklabels=failure_encoder.classes_,
            cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

sample = pd.DataFrame([{
    'Air temperature [K]': 295.6,
    'Process temperature [K]': 309.2,
    'Rotational speed [rpm]': 1380,
    'Torque [Nm]': 48.7,
    'Tool wear [min]': 165,
    'Type': label_encoders['Type'].transform(['H'])[0],
    'Product ID': label_encoders['Product ID'].transform(['M1485'])[0],
    'Target': 1
}])

prediction = rf.predict(sample)
predicted_label = failure_encoder.inverse_transform(prediction)
print("Predicted Failure Type:", predicted_label[0])

joblib.dump(rf, 'rf_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(failure_encoder, 'failure_encoder.pkl')
